<a href="https://colab.research.google.com/github/RCreation-92/RCreation-92/blob/main/Single_Site_Docuementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The 1st section breaks down the data that is required to run the program, and the functions used to extract data from the json file

An important part to note is that there are 2 ways to obtain data. One is from Node.JS and another is configured within initialization_module.py

To get the Node.JS data, uncomment the PyFlask functions and indent the rest of the syntax under the function get_node_data():

To get the localized data in initialization_module.py, comment out the PyFlask functions and unindent the syntax under get_node_data() function.

In [ ]:
#Calculate the time required to run the single site files.

#Getting the start time to calculate the interval

start_time = time.time()

#Getting the data from Node.JS in a json file format

request_data = request.get_json()

#Passes data from the testing scenario in initialization_module.py
#To comment out when using the data from Node.JS

received_data = get_received_data()

#The following information is obtained from the receive data function

#Getting the SKU names, and initial time for planning from the received data function
SKU_names = get_SKU_names(received_data)
init_time = get_init_time(received_data)

#Lost sales model indicate whether it is a backorder or a lost sales type of inventory planning
# To activate backordering mode, set this as 1
# For lost sales mode, set this as 2
lost_sales_mode = get_lost_sales_mode(received_data)

#Safety stock is a threshold to store a certain amount of inventory over the planning horizon

safety_stock = get_safety_stock(received_data)

#The indent, shortage, holding and wastage cost is defined in received data
# This is used to calculate the total cost of inventory planning over the planning horizon

indent_order_cost = get_indent_order_cost(received_data)
shortage_cost_list = get_shortage_cost_list(received_data)
holding_cost_list = get_holding_cost_list(received_data)
wastage_cost_list = get_wastage_cost_list(received_data)

#Supplier info includes the cost that they are offering for the specific SKU

supplier_info = get_supplier_info(received_data)

# The minimum order quantity is the lowest amount of goods we can order, if the order is not 0

minimum_order_quantities, minimum_order_quantities_unconstrained = get_minimum_order_quantities(received_data)

#Ordering horizon is the total number of periods that is configured in the received data

ordering_horizon = get_ordering_horizon(received_data)

num_demand_scenarios = 1

#Demand list is the list of quantities and probabilities associated with the quantities that are expected
#to occur in a given month

demand_list = get_demand_list(received_data, num_demand_scenarios)

# The initial inventory is the starting quantity of SKU that we have at init time

request_initial_inventory = get_request_initial_inventory(received_data)

# The number of shuffles, the start time and number of variations in the stochasticity
# is done here. If the numbers are increased, the computational speed is decreased

from_time = 0
num_scenarios = 1
num_shuffles = 1
num_scenarios_param_variations = 1


# Obtain the minimum lead time from each supplier
# calculate the number of iterations that needs to be done before terminating the algorithm

min_lead_time = get_min_lead_time(supplier_info)
init_time_up_to = (ordering_horizon['duration']-1) - (min(min_lead_time) + 1)




The 2nd part describes how we arrange the data for the optimization module. It arranges the data in such a way for the optimization module to understand and produce the results that we want.

In [ ]:


for init_time in range(init_time_up_to+1):

  #From the range of the init time to the calculated iterations needed to complete the algorithm,
  #we obtain the open orders from the received data, supplier info, and generate the demand scenarios from the demand list
    avg_cost = dict()
    # cost_at_scenario = dict()
    order_quant_avg = dict()
    request_open_order = get_open_order_file(init_time, get_open_order_at_t0(received_data))
    open_order = rearranged_open_orders(init_time, request_open_order)
    process_variables = get_process_variables(open_order, supplier_info)
    possible_scenarios = {scenario: generate_scenarios(scenario,
                                                                init_time,
                                                                process_variables,
                                                                num_scenarios_param_variations,
                                                                ordering_horizon,
                                                                open_order) for scenario in range(num_scenarios)}


  # In this part, we open a dictionary for the possible scenarios and their cost
    possible_scenarios_moq = {scenario: dict() for scenario in range(num_scenarios)}
    cost_at_scenario = {scenario: dict() for scenario in range(num_scenarios)}

   # open a dictionary to store the recommended orders
    order_recommended = dict()
    
    # Within the dictionary, write a nested dictionary to store the sampled scenarios for each demand sample generated

    for scenario in range(num_scenarios):
        order_recommended[scenario] = dict()
        for sample in possible_scenarios[scenario]:
            order_recommended[scenario][sample] = dict()
            for demand_sample in demand_list.keys():

              # Get the initial inventory from the requested data
                print("Solving for:")
                print("init_time ", init_time, ", scenario ", scenario, " sample ", sample)
                # print('Obtained supply pattern ->', possible_scenarios[scenario][sample])
                initial_inventory_0 = get_initial_inventory(from_time, request_initial_inventory, scenario)
                # print('req 2', initial_inventory_0)

              # Calculate the total initial inventory after adding the initial inventory and open orders, subtracted by the demand
              # for each scenario
                initial_inventory = compute_initial_inventory(init_time,
                                                              initial_inventory_0,
                                                              possible_scenarios[scenario][sample],
                                                              process_variables,
                                                              demand_list[demand_sample],
                                                              ordering_horizon,
                                                              lost_sales_mode,
                                                              scenario)

            # Pass the information: Init_time, suppliers list, ordering horizon, shelf life, lead time and cost
            # pass the additional information such as minimum order quantities and safety stock to the optimization engine

            # After callng the InventorySolution class, call the functions generate_x_y_delta_index_list, get_earliest_arrival and
            # compute_optimal_solution
                inventory_instance = InventorySolution(
                                                        init_time,
                                                        process_variables['suppliers_list'],
                                                        range(ordering_horizon['duration']),
                                                        range(ordering_horizon['duration']),
                                                        ordering_horizon['duration'],
                                                        process_variables['shelf_life_range'],
                                                        process_variables['lead_time_distribution'],
                                                        process_variables['negative_periods'],
                                                        indent_order_cost,
                                                        holding_cost_list,
                                                        wastage_cost_list,
                                                        minimum_order_quantities_unconstrained,
                                                        safety_stock
                                                    )

                inventory_instance.generate_x_y_delta_index_list()
                inventory_instance.get_earliest_arrival(possible_scenarios[scenario][sample])

                # This function passes the information to the optimization engine
                inventory_instance.compute_optimal_solution(
                                                            initial_inventory,
                                                            demand_list[demand_sample],
                                                            possible_scenarios[scenario][sample]
                                                            )
              # We obtain the recommended orders from this function, after the engine has processed it
                order_recommended[scenario][sample][demand_sample] = inventory_instance.get_order_list(possible_scenarios[scenario][sample])

                msg = 'Recommended order[' + str(sample)  + ']['+ str(demand_sample) + '] for init time -> ' + str(init_time) + '\n ->' + str(order_recommended[scenario][sample][demand_sample])
                logging.info(msg)
                # order_recommended[scenario][sample][demand_sample] --> dispatch_period key error

              # A secondary function is run to compute the recommended orders after considering the MOQ
                possible_scenarios_moq[scenario][demand_sample] = compute_cost_with_MOQ(order_recommended[scenario][sample][demand_sample],
                                                                                        init_time, possible_scenarios[scenario][sample],
                                      process_variables,
                                      ordering_horizon, indent_order_cost,
                                      holding_cost_list, wastage_cost_list,
                                      minimum_order_quantities, safety_stock, initial_inventory, demand_list,
                                      initial_inventory_0, from_time, shortage_cost_list,
                                      num_shuffles, demand_sample)

    # msg = 'Avg cost ->' + str(avg_cost)
    # logging.info(msg)
  #The average cost is calculated after considering all the scenarios and taking the best possible scenario
  # The best possible scenario is selected by iteration if possible_scenarios_moq[scenario][indx]['avg_cost'] < min_cost
    min_indx = 0
    min_scenario = 0
    min_cost = possible_scenarios_moq[min_scenario][min_indx]['avg_cost']
    for scenario in range(num_scenarios):
        for indx in possible_scenarios_moq[scenario]:
            if possible_scenarios_moq[scenario][indx]['avg_cost'] < min_cost:
                min_scenario = scenario
                min_indx = indx
                min_cost = possible_scenarios_moq[scenario][indx]['avg_cost']
    # msg = 'minimum cost obtained at -> ' + str(min_indx) + '\n' + str(possible_scenarios[min_indx])
    # logging.info(msg)
    # msg = 'order_placed_here at time-> ' + str(init_time) + '\n' + str(order_recommended[min_indx][0])
    # logging.info(msg)

    # After the best scenario is selected, we format the quantity ordered into a dataframe for the next part.
    # The next part will determine the total costs accumulated over the planning horizon.
    quantity_ordered = place_order(
                                    init_time,
                                    possible_scenarios_moq[min_scenario][min_indx]['order_recommended_moq'],
                                    process_variables['suppliers_list']
                                  )

    print(quantity_ordered)

The next portion's objective is in 2 parts: Calculating the total costs accumulated over the planning horizon, and formatting the results for Node.JS to send back to the front-end dashboard.

In [ ]:
#=======================================================================================================================
    # Test part
    #=======================================================================================================================

  # Generate the cost average and structured results dictionary
    cost_average = dict()
    result_struct = dict()
    if init_time >= init_time_up_to:
        time0 = 0
        num_test_scenarios = 100
        generated_test_scenarios = generate_test_scenarios(process_variables,
                                                        num_test_scenarios,
                                                        ordering_horizon,
                                                        init_time_up_to)

  # Seperate the costs into the order cost, holding cost, wastage cost and shortage cost.

        cost_average['order_cost'] = dict()
        cost_average['holding_cost'] = dict()
        cost_average['wastage_cost'] = dict()
        cost_average['shortage_cost'] = dict()
        on_hand_qty_dict = dict()

# Compute cost over the scenarios according to the recommended quantities
# Total cost is the cost over all periods
# Order cost is the cost of ordering all the quantities over the planning horizon
# Holding cost is calculated by determining how much inventory is left over each period
# Shortage cost is calculated by determining how much demand is unfulfilled
# Wastage cost is calculated by determining how much inventory has expired in each period

        scenario_results = dict()
        for scenario in generated_test_scenarios:
            for demand_scenario in demand_list:
                total_cost, order_cost, wastage_cost, holding_cost, shortage_cost, wastage_qty, on_hand_qty = compute_cost(
                    get_initial_inventory(from_time, request_initial_inventory, scenario),
                    generated_test_scenarios[scenario],
                    demand_list[demand_scenario],
                    indent_order_cost,
                    holding_cost_list,
                    wastage_cost_list,
                    process_variables['shelf_life_range'],
                    time0,
                    ordering_horizon['duration'],
                    process_variables['suppliers_list'],
                    shortage_cost_list
                )

                #Generate the output results list and structure it for Node.JS to read
                # First we obtain the costs per unit
                output_results_list = dict()
                output_results_list['order_cost'] = order_cost
                output_results_list['wastage_cost'] = wastage_cost
                output_results_list['holding_cost'] = holding_cost
                output_results_list['shortage_cost'] = shortage_cost
                on_hand_qty_dict['quantity'] = on_hand_qty.values()
                scenario_results[scenario] = output_results_list

                msg = 'scenario results ->' + str(scenario_results[scenario])
                logging.info(msg)

                # Obtain the costs and structure it
                if scenario == 0 and demand_scenario == 0:
                    cost_average['order_cost'] = output_results_list['order_cost']
                    cost_average['wastage_cost'] = output_results_list['wastage_cost']
                    cost_average['holding_cost'] = output_results_list['holding_cost']
                    cost_average['shortage_cost'] = output_results_list['shortage_cost']
                else:

                  # Iterate the costs and add it for each period in the planning horizon.
                    for index in output_results_list['order_cost']:
                        cost_average['order_cost'][index] += output_results_list['order_cost'][index]
                    for index in output_results_list['wastage_cost']:
                        cost_average['wastage_cost'][index] += output_results_list['wastage_cost'][index]
                    for index in output_results_list['holding_cost']:
                        cost_average['holding_cost'][index] += output_results_list['holding_cost'][index]
                    for index in output_results_list['shortage_cost']:
                        cost_average['shortage_cost'][index] += output_results_list['shortage_cost'][index]
      # Since all scenarios have been considered, we need to divide it by the number of scenarios that we have generated.
        for index in cost_average['order_cost']:
            cost_average['order_cost'][index] = cost_average['order_cost'][index] / (num_test_scenarios*num_demand_scenarios)
        for index in cost_average['wastage_cost']:
            cost_average['wastage_cost'][index] = cost_average['wastage_cost'][index] / (num_test_scenarios*num_demand_scenarios)
        for index in cost_average['holding_cost']:
            cost_average['holding_cost'][index] = cost_average['holding_cost'][index] / (num_test_scenarios*num_demand_scenarios)
        for index in cost_average['shortage_cost']:
            cost_average['shortage_cost'][index] = cost_average['shortage_cost'][index] / (num_test_scenarios*num_demand_scenarios)
        msg = 'check cost average ->' + str(cost_average)
        logging.info(msg)
        write_cost_average(cost_average)

        # get_structured_results arranges all the relevant data into a proper json format
        for time_at in range(0, ordering_horizon['duration']):
            if time_at > 0:
                result_struct[time_at] = get_structured_result(received_data["input"]["RequestKey"],
                                                               received_data["input"]["PlanDate"],
                                                               received_data["input"]["PlanningHorizon"],
                                                               received_data["input"]["Warehouse"],
                                                               received_data["input"]["Items"][0]["SKUCode"],
                                                               time_at,
                                                               process_variables['suppliers_list'],
                                                               process_variables['shelf_life_range'],
                                                               cost_average['order_cost'][time_at],
                                                               cost_average['holding_cost'][time_at + 1],
                                                               cost_average['wastage_cost'][time_at + 1],
                                                               cost_average['shortage_cost'][time_at],
                                                               quantity_ordered,
                                                               indent_order_cost,
                                                               shortage_cost_list
                                                               )

            else:

            # This condition is for the rest of the time periods
                shortage_cost_at_time0 = 0.0
                # print('cost_average', cost_average['order_cost'])
                result_struct[time_at] = get_structured_result(received_data["input"]["RequestKey"],
                                                               received_data["input"]["PlanDate"],
                                                               received_data["input"]["PlanningHorizon"],
                                                               received_data["input"]["Warehouse"],
                                                               received_data["input"]["Items"][0]["SKUCode"],
                                                               time_at,
                                                               process_variables['suppliers_list'],
                                                               process_variables['shelf_life_range'],
                                                               cost_average['order_cost'][time_at],
                                                               cost_average['holding_cost'][time_at + 1],
                                                               cost_average['wastage_cost'][time_at + 1],
                                                               shortage_cost_at_time0,
                                                               quantity_ordered,
                                                               indent_order_cost,
                                                               shortage_cost_list

                                                               )
    print('result_struct', result_struct)
    # msg = 'result_struct' + str(result_struct)
    # logging.info(msg)
    # result_struct_main["Main"] = result_struct
    # return jsonify(result_struct_main)


    # print('result_struct ->', result_struct[init_time])

end_time = time.time()

time_different = end_time - start_time